In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.chrome.options import Options
import time

In [2]:
k=pd.read_csv("가게_이름_링크.csv")
k

,음식점,링크
0,하이레,m.place.naver.com/restaurant/1083519696/home
1,윤스쿡,m.place.naver.com/restaurant/166240712/home
2,화로상회 광운대본점,m.place.naver.com/restaurant/37611559/home
3,진미통닭,m.place.naver.com/restaurant/1401137405/home
4,푸른스시 광운대본점,m.place.naver.com/restaurant/1769424895/home
...,...,...
70,온량,m.place.naver.com/restaurant/1796729449/home
71,짜카,m.place.naver.com/restaurant/1044069286/home
72,서민스시 장위 본점,m.place.naver.com/restaurant/1820940739/home
73,더원,m.place.naver.com/restaurant/1092067499/home


In [2]:
kakao="https://map.kakao.com/"
naver="map.naver.com"

In [ ]:
all_df=pd.DataFrame()

In [194]:
options = Options()



#options.add_argument("--headless")
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)
driver.get(kakao)

driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="37f26a101294f1fe271bc8dd1516bd5a")>

In [195]:
num=1
#후기더보기 누르기 버튼
while(num<100):
    
    try:
        num+=1
        driver.find_element(By.CLASS_NAME,"txt_more").click()


    #오류가 뜨거나 후기접기가 나타날 때까지 후기 더보기 누르기
    #더이상 txt_more이라는 element없으면 break
    except:
        break
time.sleep(2)

evaluation_review=driver.find_element(By.CLASS_NAME,"evaluation_review")


unit_info_list=evaluation_review.find_elements(By.CLASS_NAME,"unit_info")
star_info_list=evaluation_review.find_elements(By.CLASS_NAME,"star_info")
#wrap_likepoint_list=evaluation_review.find_elements(By.CLASS_NAME,"wrap_likepoint")
comment_info_list=evaluation_review.find_elements(By.CLASS_NAME,"comment_info")

#리뷰 전체 리스트로 가져오기
cuisine_evaluation=driver.find_element(By.CLASS_NAME,"list_evaluation")

time.sleep(2)
#리뷰 리스트화
cuisine_evaluation_list=cuisine_evaluation.find_elements(By.TAG_NAME,"li")

id_list=[]

for i in range(len(cuisine_evaluation_list)):
    id_list.append(cuisine_evaluation_list[i].get_attribute("data-userid"))
    
id_list=list(filter(None, id_list))


all_review=[]
#빈 리스트 생성
for i in range(len(id_list)):
    user=[]
    title=driver.find_element(By.CLASS_NAME,"inner_place").find_element(By.CLASS_NAME,"tit_location").text
    user.append(title)
    time.sleep(1)
    
    user.append(id_list[i])
    
    
    #해당 리뷰 작성자의 개인정보
    nickname=unit_info_list[i].find_element(By.CLASS_NAME,"txt_username").text
    user.append(nickname)
    badge=unit_info_list[i].find_element(By.CLASS_NAME,"badge_info").text[:5]
    user.append(badge)
    user_review_count=unit_info_list[i].find_elements(By.CLASS_NAME,"txt_desc")[0].text
    user.append(user_review_count)
    user_rating_average=unit_info_list[i].find_elements(By.CLASS_NAME,"txt_desc")[1].text
    user.append(user_rating_average)

    #평점
    rating=int(star_info_list[i].find_element(By.CLASS_NAME,"inner_star").get_attribute("style")[7:9])/20
    user.append(rating)

    # try:
    #     #사용자가 남긴 태그를 리스트로 저장
    #     recommend_point=wrap_likepoint_list[i].find_element(By.CLASS_NAME,"group_likepoint").text.split("\n")
    #     user.append(recommend_point)
    # except:
    #     print()

    #사용자가 남긴 리뷰를 리스트로 저장.
    review_comment=comment_info_list[i].find_element(By.TAG_NAME,"span").text.split("\n")
    user.append(review_comment)
    
    all_review.append(user)

In [196]:
len(all_review)

403

In [197]:
tmp_df=pd.DataFrame(data=all_review)
tmp_df=tmp_df.drop_duplicates([0,1])
all_df=pd.concat([all_df,tmp_df])

In [198]:
all_df.replace(0.5, 5.0, inplace=True)

In [199]:
all_df=all_df.drop_duplicates([0,1])

In [201]:
all_df

,0,1,2,3,4,5,6,7
0,홍대조폭떡볶이 홍대2호점,481524804,박지범,브론즈 배,5,3.6,3.0,[전에 비하면 가격이 오르긴 했어도 다르곳에 비하면 저렴한 가격에 먹을수있음.]
1,홍대조폭떡볶이 홍대2호점,1254054279,상상용,실버 배지,167,3.4,2.0,[다른분식집에 비해 맛이 너무없어서 실망스러웠습니다]
2,홍대조폭떡볶이 홍대2호점,764377187,가시멸치,골드 배지,220,3.3,4.0,[조폭떡볶이와 순대(내장x). 김밥이 먹고싶었는데 항상 저녁에 제일 먼저 떨어진다고...
3,홍대조폭떡볶이 홍대2호점,556600675,🕓,실버 배지,34,3.8,5.0,[]
4,홍대조폭떡볶이 홍대2호점,484323118,sky,브론즈 배,11,2.8,5.0,[여기 직원분들 바뀌고나서 친절하셔서 너무 좋아요~떡볶이는 말할것도 없고 특히 순대...
...,...,...,...,...,...,...,...,...
278,산수갑산 을지로점,770338845,둥,골드 배지,214,3.4,1.0,[]
279,산수갑산 을지로점,779388676,IIIllIllI,골드 배지,351,3.6,2.0,[굳에 말하자면 맛없음에 가깝다.]
280,산수갑산 을지로점,1359458728,이종민,브론즈 배,6,4.7,4.0,"[누린 내 없는 구수하고 진한 국물, 순대도 푸짐, 술을 술술 부르는 술국]"
281,산수갑산 을지로점,445500302,한,실버 배지,41,1.6,1.0,[순대는 식어서 나오는데 돼지비린내는 엄청나고;; 따뜻한 국 용기는 플라스틱이고 국...


In [203]:
all_df.to_csv('tmp_data.csv',index=False)

In [151]:
def crawlcrawl(e):

    #상세정보 창 열기
    cuisine_1_detail=e.find_element(By.CLASS_NAME,"moreview")
    cuisine_1_detail.click()
    time.sleep(2)

    #작동되는 페이지 옮기기
    driver.switch_to.window(driver.window_handles[-1])
    time.sleep(2)
        
    num=1
    #후기더보기 누르기 버튼
    while(num<100):
        
        try:
            num+=1
            driver.find_element(By.CLASS_NAME,"txt_more").click()


        #오류가 뜨거나 후기접기가 나타날 때까지 후기 더보기 누르기
        #더이상 txt_more이라는 element없으면 break
        except:
            break
    time.sleep(2)
    
    evaluation_review=driver.find_element(By.CLASS_NAME,"evaluation_review")
    

    unit_info_list=evaluation_review.find_elements(By.CLASS_NAME,"unit_info")
    star_info_list=evaluation_review.find_elements(By.CLASS_NAME,"star_info")
    #wrap_likepoint_list=evaluation_review.find_elements(By.CLASS_NAME,"wrap_likepoint")
    comment_info_list=evaluation_review.find_elements(By.CLASS_NAME,"comment_info")
    
    #리뷰 전체 리스트로 가져오기
    cuisine_evaluation=driver.find_element(By.CLASS_NAME,"list_evaluation")

    time.sleep(2)
    #리뷰 리스트화
    cuisine_evaluation_list=cuisine_evaluation.find_elements(By.TAG_NAME,"li")
    
    id_list=[]
    
    for i in range(len(cuisine_evaluation_list)):
        id_list.append(cuisine_evaluation_list[i].get_attribute("data-userid"))
        
    id_list=list(filter(None, id_list))
    
    
    all_review=[]
    #빈 리스트 생성
    for i in range(len(id_list)):
        user=[]
        title=driver.find_element(By.CLASS_NAME,"inner_place").find_element(By.CLASS_NAME,"tit_location").text
        user.append(title)
        time.sleep(1)
        
        user.append(id_list[i])
        
        
        #해당 리뷰 작성자의 개인정보
        nickname=unit_info_list[i].find_element(By.CLASS_NAME,"txt_username").text
        user.append(nickname)
        badge=unit_info_list[i].find_element(By.CLASS_NAME,"badge_info").text[:5]
        user.append(badge)
        user_review_count=unit_info_list[i].find_elements(By.CLASS_NAME,"txt_desc")[0].text
        user.append(user_review_count)
        user_rating_average=unit_info_list[i].find_elements(By.CLASS_NAME,"txt_desc")[1].text
        user.append(user_rating_average)
    
        #평점
        rating=int(star_info_list[i].find_element(By.CLASS_NAME,"inner_star").get_attribute("style")[7:9])/20
        user.append(rating)
    
        # try:
        #     #사용자가 남긴 태그를 리스트로 저장
        #     recommend_point=wrap_likepoint_list[i].find_element(By.CLASS_NAME,"group_likepoint").text.split("\n")
        #     user.append(recommend_point)
        # except:
        #     print()
    
        #사용자가 남긴 리뷰를 리스트로 저장.
        review_comment=comment_info_list[i].find_element(By.TAG_NAME,"span").text.split("\n")
        user.append(review_comment)
        
        all_review.append(user)
    
    
    tmp_df=pd.DataFrame(data=all_review)
    #새로 연창 종료하고 원래 창으로 드라이버 주체 바꾸기
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    
    
    
    return tmp_df


In [8]:
all_df=pd.DataFrame()

In [9]:
for i in kakao_cuisine_1:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])

In [11]:
all_df

,0,1,2,3,4,5,6,7
0,엄마마늘보쌈,1366643171,다이어트포기,실버 배지,50,3.9,4.0,"[보쌈은 너무맛있어서 공간 협소한건 생각도 안남, 김치맛 복불복]"
1,엄마마늘보쌈,2290116892,미식견,실버 배지,34,4.2,4.0,[보쌈정식 먹었는데 돼지잡내없이 맛있고 밑반찬도 먹을만해요 마늘보쌈도 맛있어요]
2,엄마마늘보쌈,312224650,서재형,실버 배지,13,5.0,0.5,"[엄마마늘보쌈은 석계역 보쌈집 터줏대감, 입니다.]"
3,엄마마늘보쌈,809029638,서아아빠,실버 배지,22,4.9,0.5,"[맛나여...., 맛이란게..., 서로 약간씩 차이가 있으니까..., 노포분위기 좋..."
4,엄마마늘보쌈,1639004379,흐리거나비,실버 배지,24,4.1,3.0,[기본으로 주는 전은 맛있었다. 보쌈도 나쁘지 않았다. 근데 굳이 갈까...]
...,...,...,...,...,...,...,...,...
24,서민초밥,566797733,이영우,브론즈 배,5,3.8,0.5,[]
0,1일1잔,379177948,흐어어엉,실버 배지,57,4.2,0.5,[마카롱 하나에 이천원밖에 안하는데 엄청 맛있어요 달지 않은 마카롱이라 부담없이 먹...
1,1일1잔,512472443,삐롱이,실버 배지,47,4.1,0.5,"[사장님이 친절하시고 가격도 저렴해요 , 마카롱 꼬끄는 쫀득쫀득이아니라 쫜득쫜득해요..."
2,1일1잔,1073182616,*_*,실버 배지,33,3.8,0.5,[달지않은 마카롱 최고⚘]


In [17]:
data=all_df.drop_duplicates([0,1])

In [18]:
data

,0,1,2,3,4,5,6,7
0,엄마마늘보쌈,1366643171,다이어트포기,실버 배지,50,3.9,4.0,"[보쌈은 너무맛있어서 공간 협소한건 생각도 안남, 김치맛 복불복]"
1,엄마마늘보쌈,2290116892,미식견,실버 배지,34,4.2,4.0,[보쌈정식 먹었는데 돼지잡내없이 맛있고 밑반찬도 먹을만해요 마늘보쌈도 맛있어요]
2,엄마마늘보쌈,312224650,서재형,실버 배지,13,5.0,0.5,"[엄마마늘보쌈은 석계역 보쌈집 터줏대감, 입니다.]"
3,엄마마늘보쌈,809029638,서아아빠,실버 배지,22,4.9,0.5,"[맛나여...., 맛이란게..., 서로 약간씩 차이가 있으니까..., 노포분위기 좋..."
4,엄마마늘보쌈,1639004379,흐리거나비,실버 배지,24,4.1,3.0,[기본으로 주는 전은 맛있었다. 보쌈도 나쁘지 않았다. 근데 굳이 갈까...]
...,...,...,...,...,...,...,...,...
24,서민초밥,566797733,이영우,브론즈 배,5,3.8,0.5,[]
0,1일1잔,379177948,흐어어엉,실버 배지,57,4.2,0.5,[마카롱 하나에 이천원밖에 안하는데 엄청 맛있어요 달지 않은 마카롱이라 부담없이 먹...
1,1일1잔,512472443,삐롱이,실버 배지,47,4.1,0.5,"[사장님이 친절하시고 가격도 저렴해요 , 마카롱 꼬끄는 쫀득쫀득이아니라 쫜득쫜득해요..."
2,1일1잔,1073182616,*_*,실버 배지,33,3.8,0.5,[달지않은 마카롱 최고⚘]


In [19]:
data=data[[0,6]]
data.columns=['가게','평점']
tmp=data.groupby('가게')['평점'].mean().to_frame()

In [20]:
tmp

,평점
가게,
1일1잔,0.500000
그레도제빵사,1.632576
남송통닭,1.250000
남해바다마차,1.830000
무명칼국수,1.764706
서민초밥,1.100000
엄마마늘보쌈,1.666667
엉터리양평해장국 석계역점,2.333333
연어시장 석계본점,1.200000


In [207]:
all_df.to_csv("textmining_review_kakao_page1.csv")

In [195]:
tmp=crawlcrawl(kakao_cuisine_1[0])

In [65]:
all_df

,0,1,2,3,4,5,6,7
0,화로상회 광운대점,1825213275,냠냠쩝쩝박사의 맛집탐방,실버 배지,92,3.5,4.0,"[맛5 가격5 매장크기3 친절함3 분위기4 위생3, 내 기준 고기 무한리필 원탑, ..."
1,화로상회 광운대점,1071880997,히히,브론즈 배,9,4.0,0.5,"[너무 저렴해서 걱정했는데, 너무 맛있게 먹고 나왔습니다:)]"
2,화로상회 광운대점,1034272868,진실만을말하는 개굴,실버 배지,34,4.0,0.5,[역시 본점클라스... 존맛. 다만 맛있고 가성비 좋은만큼 웨이팅 각오하셔야됨. 다...
3,화로상회 광운대점,1363274427,이재원,실버 배지,53,3.5,3.0,[불 약한 자리에 앉으면…]
4,화로상회 광운대점,1289095506,고양이좋아,실버 배지,94,4.2,0.5,[광운대 아이스링크 갔다가 먹으면 완벽]
...,...,...,...,...,...,...,...,...
1,남송통닭,316219944,(알 수 없음),실버 배지,67,1.1,1.0,"[픽업하여 먹어본 사람 으로 솔직히 평가함., 가격이 사진 상 가격보다 3천원이 더..."
2,남송통닭,510031137,Gerald B.,골드 배지,188,3.8,4.0,[생활의달인 909회]
3,남송통닭,1249726533,아람,브론즈 배,4,1.0,1.0,"[석계역 거주 30년 넘었는데, 생활의 달인에 나와서 반가운 나머지 다음날 바로 갔..."
4,남송통닭,256259466,히츠 앰버,실버 배지,22,4.5,0.5,[월계동 이사온지 몇 년 됐는데 여기 통닭집이 가장 맛있습니다. 저렴하기도 하구요.]


In [58]:
driver.find_element(By.ID,"info.search.page.no2").click()
time.sleep(2)
kakao_cuisine_2=driver.find_elements(By.CLASS_NAME,"PlaceItem")
for i in kakao_cuisine_2:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

In [59]:
driver.find_element(By.ID,"info.search.page.no3").click()
time.sleep(2)
kakao_cuisine_3=driver.find_elements(By.CLASS_NAME,"PlaceItem")
for i in kakao_cuisine_3:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

In [ ]:
driver.find_element(By.ID,"info.search.page.no4").click()
time.sleep(2)
kakao_cuisine_4=driver.find_elements(By.CLASS_NAME,"PlaceItem")
for i in kakao_cuisine_4:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

In [61]:
driver.find_element(By.ID,"info.search.page.no5").click()
time.sleep(2)
kakao_cuisine_5=driver.find_elements(By.CLASS_NAME,"PlaceItem")
for i in kakao_cuisine_5:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

In [62]:
driver.find_element(By.ID,"info.search.page.next").click()
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no1").click()
time.sleep(2)

kakao_cuisine_6 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_6:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

<selenium.webdriver.remote.webelement.WebElement (session="961f82f4d25bc7f7b6b482f27c2628f9", element="f.7947FF88CF08BC772977B046A9A309E5.d.1B8C9DE51B714A0549F9BC9018FA670E.e.26705")>

In [77]:
all_df_1_6=all_df

In [78]:
all_df_1_6.to_csv("df_1_6.csv")

카페피버 중복 값 제거하기
user-id랑 가게이름으로 중복값 제거하면 될듯.

In [82]:
kakao_cuisine_7 = driver.find_elements(By.CLASS_NAME, "PlaceItem")


In [85]:
kakao_cuisine_7[1].find_element(By.CLASS_NAME,"numberofscore").text

'0건'

In [87]:
driver.find_element(By.ID, "info.search.page.no2").click()
time.sleep(2)

kakao_cuisine_7 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_7:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)


In [93]:
kakao_cuisine_8 = driver.find_elements(By.CLASS_NAME, "PlaceItem")


In [96]:
kakao_cuisine_8[0].text

'즐겨찾기\n로드뷰\nA 사계절김밥 분식\n4.8\n4건\n리뷰 5\n서울 노원구 광운로 72\n(지번) 월계동 379-40\n휴무일\n화요일\n02-941-7789\n상세보기'

In [97]:
driver.find_element(By.ID, "info.search.page.no3").click()
time.sleep(2)

kakao_cuisine_8 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_8:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no4").click()
time.sleep(2)



In [98]:
kakao_cuisine_9 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_9:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no5").click()
time.sleep(2)


In [99]:

kakao_cuisine_10 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_10:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)



In [100]:
all_df

,0,1,2,3,4,5,6,7
0,화로상회 광운대점,1825213275,냠냠쩝쩝박사의 맛집탐방,실버 배지,92,3.5,4.0,"[맛5 가격5 매장크기3 친절함3 분위기4 위생3, 내 기준 고기 무한리필 원탑, ..."
1,화로상회 광운대점,1071880997,히히,브론즈 배,9,4.0,0.5,"[너무 저렴해서 걱정했는데, 너무 맛있게 먹고 나왔습니다:)]"
2,화로상회 광운대점,1034272868,진실만을말하는 개굴,실버 배지,34,4.0,0.5,[역시 본점클라스... 존맛. 다만 맛있고 가성비 좋은만큼 웨이팅 각오하셔야됨. 다...
3,화로상회 광운대점,1363274427,이재원,실버 배지,53,3.5,3.0,[불 약한 자리에 앉으면…]
4,화로상회 광운대점,1289095506,고양이좋아,실버 배지,94,4.2,0.5,[광운대 아이스링크 갔다가 먹으면 완벽]
...,...,...,...,...,...,...,...,...
0,하남돼지집 석계점,2791529822,박서희,브론즈 배,1,5.0,0.5,"[으휴..여기 별1개 주고 리뷰다신분들 뭐 어디 고기전문가들만 모이셨나 , 그냥 맘..."
1,하남돼지집 석계점,3350576225,월계동먹보,브론즈 배,1,1.0,1.0,[돈값을 못함. 프랜차이즈라 기대 했는데 비싸기만 하고 평범은 커녕 오히려 별로. ...
2,하남돼지집 석계점,2290116892,미식견,실버 배지,21,4.3,1.0,[고기 너무 못굽고 질기고 맛없음 직원교육 필요]
3,하남돼지집 석계점,699581225,흠흠신서,브론즈 배,2,1.0,1.0,"[기름 겁나 튀고, 고기에서 냄새남. 어떤건 덜 익혀서 질기고, 어떤건 태워서 딱딱..."


In [101]:
driver.find_element(By.ID, "info.search.page.next").click()
time.sleep(2)


driver.find_element(By.ID, "info.search.page.no1").click()
time.sleep(2)

kakao_cuisine_11 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_11:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)


In [102]:

driver.find_element(By.ID, "info.search.page.no2").click()
time.sleep(2)

kakao_cuisine_12 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_12:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no3").click()
time.sleep(2)


In [103]:

kakao_cuisine_13 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_13:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no4").click()
time.sleep(2)


In [104]:

kakao_cuisine_14 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_14:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no5").click()
time.sleep(2)


In [105]:

kakao_cuisine_15 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_15:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.next").click()
time.sleep(2)

In [107]:

kakao_cuisine_16 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_16:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no2").click()
time.sleep(2)

In [108]:

kakao_cuisine_17 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_17:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no3").click()
time.sleep(2)

In [109]:

kakao_cuisine_18 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_18:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no4").click()
time.sleep(2)

In [110]:

kakao_cuisine_19 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_19:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no5").click()
time.sleep(2)

In [111]:

kakao_cuisine_20 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_20:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    if i.find_element(By.CLASS_NAME,"numberofscore").text=="0건":
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.next").click()
time.sleep(2)

In [112]:
all_df

,0,1,2,3,4,5,6,7
0,화로상회 광운대점,1825213275,냠냠쩝쩝박사의 맛집탐방,실버 배지,92,3.5,4.0,"[맛5 가격5 매장크기3 친절함3 분위기4 위생3, 내 기준 고기 무한리필 원탑, ..."
1,화로상회 광운대점,1071880997,히히,브론즈 배,9,4.0,0.5,"[너무 저렴해서 걱정했는데, 너무 맛있게 먹고 나왔습니다:)]"
2,화로상회 광운대점,1034272868,진실만을말하는 개굴,실버 배지,34,4.0,0.5,[역시 본점클라스... 존맛. 다만 맛있고 가성비 좋은만큼 웨이팅 각오하셔야됨. 다...
3,화로상회 광운대점,1363274427,이재원,실버 배지,53,3.5,3.0,[불 약한 자리에 앉으면…]
4,화로상회 광운대점,1289095506,고양이좋아,실버 배지,94,4.2,0.5,[광운대 아이스링크 갔다가 먹으면 완벽]
...,...,...,...,...,...,...,...,...
12,봉추찜닭 석계점,302424845,이정하,브론즈 배,14,3.9,3.0,[]
13,봉추찜닭 석계점,1301853532,주관적댓글러,브론즈 배,5,4.0,0.5,"[배달 빠름 7분거리에 35분걸림(조리시간까지), 배달료 없음!!! 사장님이 직접 ..."
14,봉추찜닭 석계점,760983481,소원,실버 배지,87,4.0,4.0,[맛은 있는데 장사안하는집같은 어두컴컴 분위기 그래도 찜닭좋아해서 가끔가요]
15,봉추찜닭 석계점,1116073766,wjap,실버 배지,535,2.5,3.0,[been]


In [113]:
all_df.to_csv("all_df.csv")

In [55]:
driver.find_element(By.ID, "info.search.page.no1").click()
kakao_cuisine_16 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_16:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no2").click()
kakao_cuisine_17 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_17:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no3").click()
kakao_cuisine_18 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_18:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no4").click()
kakao_cuisine_19 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_19:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.no5").click()
kakao_cuisine_20 = driver.find_elements(By.CLASS_NAME, "PlaceItem")
for i in kakao_cuisine_20:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df=pd.concat([all_df,tmp])
time.sleep(2)

driver.find_element(By.ID, "info.search.page.next").click()
time.sleep(2)

In [79]:
all_df.to_csv("all_df1-2.csv")

In [59]:
driver.find_element(By.ID,"info.search.page.no2").click()
for i in kakao_cuisine_2:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=124.0.6367.201); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6BA761562+60802]
	(No symbol) [0x00007FF6BA6DAC62]
	(No symbol) [0x00007FF6BA597CE4]
	(No symbol) [0x00007FF6BA59DE26]
	(No symbol) [0x00007FF6BA5A01DB]
	(No symbol) [0x00007FF6BA5A0280]
	(No symbol) [0x00007FF6BA5E6A11]
	(No symbol) [0x00007FF6BA5E6E1C]
	(No symbol) [0x00007FF6BA5DA8FC]
	(No symbol) [0x00007FF6BA60ABBF]
	(No symbol) [0x00007FF6BA5DA7C6]
	(No symbol) [0x00007FF6BA60AD90]
	(No symbol) [0x00007FF6BA62A224]
	(No symbol) [0x00007FF6BA60A923]
	(No symbol) [0x00007FF6BA5D8FEC]
	(No symbol) [0x00007FF6BA5D9C21]
	GetHandleVerifier [0x00007FF6BAA641FD+3217949]
	GetHandleVerifier [0x00007FF6BAAA6197+3488183]
	GetHandleVerifier [0x00007FF6B

In [78]:
all_df

,0,1,2,3,4,5,6,7
0,화로상회 광운대점,1825213275,냠냠쩝쩝박사의 맛집탐방,실버 배지,92,3.5,4.0,"[맛5 가격5 매장크기3 친절함3 분위기4 위생3, 내 기준 고기 무한리필 원탑, ..."
1,화로상회 광운대점,1071880997,히히,브론즈 배,9,4.0,0.5,"[너무 저렴해서 걱정했는데, 너무 맛있게 먹고 나왔습니다:)]"
2,화로상회 광운대점,1034272868,진실만을말하는 개굴,실버 배지,34,4.0,0.5,[역시 본점클라스... 존맛. 다만 맛있고 가성비 좋은만큼 웨이팅 각오하셔야됨. 다...
3,화로상회 광운대점,1363274427,이재원,실버 배지,53,3.5,3.0,[불 약한 자리에 앉으면…]
4,화로상회 광운대점,1289095506,고양이좋아,실버 배지,94,4.2,0.5,[광운대 아이스링크 갔다가 먹으면 완벽]
...,...,...,...,...,...,...,...,...
6,설빙 광운대점,1580063626,최나은,골드 배지,201,4.1,0.5,[잉 요기 좋은뎅... 다른 지점보다 양이 적은 것도 아니고 뭐 누락된 적도 없고 ...
7,설빙 광운대점,1226027760,연이,브론즈 배,2,1.0,1.0,[오래걸려서 아이스팩 넣어준다고했는데 아무것도없네요]
8,설빙 광운대점,767918895,성민제,브론즈 배,8,3.3,2.0,[떡볶이 포장했는데 심하게 터져잇네요 ㅜ]
9,설빙 광운대점,767918895,성민제,브론즈 배,8,3.3,2.0,[떡볶이 포장했는데 심하게 터져잇네요 ㅜ]


In [58]:


time.sleep(2)
driver.find_element(By.ID,"info.search.page.no2").click()
for i in kakao_cuisine_2:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

time.sleep(2)  
driver.find_element(By.ID,"info.search.page.no3").click()
for i in kakao_cuisine_3:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

time.sleep(2)
driver.find_element(By.ID,"info.search.page.no4").click()
for i in kakao_cuisine_4:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])


time.sleep(2)
driver.find_element(By.ID,"info.search.page.no5").click()
for i in kakao_cuisine_5:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

driver.find_element(By.ID,"info.search.page.next").click()
time.sleep(2)
driver.find_element(By.ID,"info.search.page.no1").click()
for i in kakao_cuisine_6:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

driver.find_element(By.ID,"info.search.page.no2").click()
for i in kakao_cuisine_7:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])
    
driver.find_element(By.ID,"info.search.page.no3").click()
for i in kakao_cuisine_8:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

driver.find_element(By.ID,"info.search.page.no4").click()
for i in kakao_cuisine_9:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])
    
driver.find_element(By.ID,"info.search.page.no5").click()
for i in kakao_cuisine_10:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

driver.find_element(By.ID,"info.search.page.next").click()
time.sleep(2)
driver.find_element(By.ID,"info.search.page.no1").click()

for i in kakao_cuisine_11:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])


driver.find_element(By.ID,"info.search.page.no2").click()
for i in kakao_cuisine_12:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

driver.find_element(By.ID,"info.search.page.no3").click()
for i in kakao_cuisine_13:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

driver.find_element(By.ID,"info.search.page.no4").click()
for i in kakao_cuisine_14:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])
    
driver.find_element(By.ID,"info.search.page.no5").click()
for i in kakao_cuisine_15:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])
    
driver.find_element(By.ID,"info.search.page.next").click()
time.sleep(2)
driver.find_element(By.ID,"info.search.page.no1").click()
for i in kakao_cuisine_16:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

driver.find_element(By.ID,"info.search.page.no2").click()
for i in kakao_cuisine_17:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

driver.find_element(By.ID,"info.search.page.no3").click()
for i in kakao_cuisine_18:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

driver.find_element(By.ID,"info.search.page.no4").click()
for i in kakao_cuisine_19:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])

driver.find_element(By.ID,"info.search.page.no5").click()
for i in kakao_cuisine_20:
    blind=i.find_element(By.CLASS_NAME,"txt_blind")
    if blind.text:
        continue
    tmp=crawlcrawl(i)
    all_df_no_1=pd.concat([all_df,tmp])


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=124.0.6367.201); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6BA761562+60802]
	(No symbol) [0x00007FF6BA6DAC62]
	(No symbol) [0x00007FF6BA597CE4]
	(No symbol) [0x00007FF6BA59DE26]
	(No symbol) [0x00007FF6BA5A01DB]
	(No symbol) [0x00007FF6BA5A0280]
	(No symbol) [0x00007FF6BA5E6A11]
	(No symbol) [0x00007FF6BA5E6E1C]
	(No symbol) [0x00007FF6BA5DA8FC]
	(No symbol) [0x00007FF6BA60ABBF]
	(No symbol) [0x00007FF6BA5DA7C6]
	(No symbol) [0x00007FF6BA60AD90]
	(No symbol) [0x00007FF6BA62A224]
	(No symbol) [0x00007FF6BA60A923]
	(No symbol) [0x00007FF6BA5D8FEC]
	(No symbol) [0x00007FF6BA5D9C21]
	GetHandleVerifier [0x00007FF6BAA641FD+3217949]
	GetHandleVerifier [0x00007FF6BAAA6197+3488183]
	GetHandleVerifier [0x00007FF6B

In [2]:
df=pd.read_csv('all_df.csv')

In [3]:
df

,Unnamed: 0,0,1,2,3,4,5,6,7
0,0,화로상회 광운대점,1825213275,냠냠쩝쩝박사의 맛집탐방,실버 배지,92,3.5,4.0,"['맛5 가격5 매장크기3 친절함3 분위기4 위생3', '내 기준 고기 무한리필 원..."
1,1,화로상회 광운대점,1071880997,히히,브론즈 배,9,4.0,0.5,"['너무 저렴해서 걱정했는데, 너무 맛있게 먹고 나왔습니다:)']"
2,2,화로상회 광운대점,1034272868,진실만을말하는 개굴,실버 배지,34,4.0,0.5,['역시 본점클라스... 존맛. 다만 맛있고 가성비 좋은만큼 웨이팅 각오하셔야됨. ...
3,3,화로상회 광운대점,1363274427,이재원,실버 배지,53,3.5,3.0,['불 약한 자리에 앉으면…']
4,4,화로상회 광운대점,1289095506,고양이좋아,실버 배지,94,4.2,0.5,['광운대 아이스링크 갔다가 먹으면 완벽']
...,...,...,...,...,...,...,...,...,...
6129,12,봉추찜닭 석계점,302424845,이정하,브론즈 배,14,3.9,3.0,['']
6130,13,봉추찜닭 석계점,1301853532,주관적댓글러,브론즈 배,5,4.0,0.5,"['배달 빠름 7분거리에 35분걸림(조리시간까지)', '배달료 없음!!! 사장님이 ..."
6131,14,봉추찜닭 석계점,760983481,소원,실버 배지,87,4.0,4.0,['맛은 있는데 장사안하는집같은 어두컴컴 분위기 그래도 찜닭좋아해서 가끔가요']
6132,15,봉추찜닭 석계점,1116073766,wjap,실버 배지,535,2.5,3.0,['been']
